# **1 - Instalar pacotes necessários**




In [8]:
%pip -q install google-genai google-adk

# **2 - Importar libs necessárias**

In [9]:
import os, requests, warnings, textwrap, sqlite3, json, re
from datetime import datetime, date
from google import genai
from google.colab import userdata
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from IPython.display import HTML, Markdown
from IPython.display import display, Markdown # Para exibir texto formatado no Colab

warnings.filterwarnings("ignore")
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_2')
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

# **3 - Funções acessórias**

In [10]:
def formata_numero(valor):
    return '{:,.2f}'.format(valor).replace(',', 'x').replace('.', ',').replace('x', '.')

def formata_data(data):
    """Formata um objeto datetime.date para o formato dd/mm/aaaa."""
    if isinstance(data, str):
        try:
            data_obj = datetime.strptime(data, '%Y-%m-%d').date()
            return data_obj.strftime('%d/%m/%Y')
        except ValueError:
            return data  # Retorna a string original em caso de erro de parsing
    elif isinstance(data, datetime):
        return data.strftime('%d/%m/%Y')
    return data

def formata_hora(hora):
    """Formata um objeto datetime.time ou string para o formato HH:MM."""
    if isinstance(hora, str):
        try:
            hora_obj = datetime.strptime(hora, '%H:%M:%S').time()
            return hora_obj.strftime('%H:%M')
        except ValueError:
            try:
                hora_obj = datetime.strptime(hora, '%H:%M').time()
                return hora_obj.strftime('%H:%M')
            except ValueError:
                return hora  # Retorna a string original em caso de erro de parsing
    elif isinstance(hora, datetime):
        return hora.strftime('%H:%M')
    return hora

def retorna_campos_cadastro_lugar():
    descricao = input("Informe a descrição do lugar: ")
    endereco = input("Informe o endereço (opcional): ")
    numero = input("Informe o número (opcional): ")
    bairro = input("Informe o bairro (opcional): ")
    cep = input("Informe o CEP (opcional): ")
    cidade = input("Informe a cidade: ")
    uf = input("Informe a UF (ex: RJ): ").upper()

    campos_dict = {
        'descricao': descricao,
        'endereco': endereco,
        'numero': numero,
        'bairro': bairro,
        'cep': cep,
        'cidade': cidade,
        'uf': uf
        }

    return campos_dict

# **4 - Funções de banco de dados**

In [11]:
def abrir_conexao_bd():
    try:
      conn = sqlite3.connect('viagem_facil.db')
      return conn
    except sqlite3.Error as e:
      print(f"Erro ao conectar ao banco de dados: {e}")
      return None

def fechar_conexao_bd(conn):
    if conn:
      return conn.close()

def comitar_bd(conn):
    if conn:
      return conn.commit()

def criar_banco_tabelas():
    conn = abrir_conexao_bd()
    if conn:
      cursor = conn.cursor()

      cursor.execute('''
          CREATE TABLE IF NOT EXISTS lugares (
              id INTEGER PRIMARY KEY AUTOINCREMENT,
              descricao TEXT NOT NULL,
              endereco TEXT,
              numero INTEGER,
              bairro TEXT,
              cep VARCHAR(10),
              cidade TEXT,
              uf VARCHAR(2),
              finalidade VARCHAR(1),
              data_cadastro DATE NOT NULL DEFAULT (date('now')),
              hora_cadastro TIME NOT NULL DEFAULT (time('now'))
            )
        ''')

      cursor.execute('''
          CREATE TABLE IF NOT EXISTS rotas (
              id INTEGER PRIMARY KEY AUTOINCREMENT,
              origem_id INTEGER NOT NULL,
              destino_id INTEGER NOT NULL,
              quantidade_km NUMERIC NOT NULL,
              link_rota TEXT NOT NULL,
              data_cadastro DATE NOT NULL DEFAULT (date('now')),
              hora_cadastro TIME NOT NULL DEFAULT (time('now')),
              FOREIGN KEY (origem_id) REFERENCES lugares(id),
              FOREIGN KEY (destino_id) REFERENCES lugares(id)
          )
      ''')

      cursor.execute('''
          CREATE TABLE IF NOT EXISTS configuracoes (
              id INTEGER PRIMARY KEY AUTOINCREMENT,
              origem_id INTEGER NOT NULL,
              destino_id INTEGER NOT NULL,
              preco_medio_uber_veiculo_km NUMERIC ,
              data_cadastro DATE NOT NULL DEFAULT (date('now')),
              hora_cadastro TIME NOT NULL DEFAULT (time('now')),
              FOREIGN KEY (origem_id) REFERENCES lugares(id),
              FOREIGN KEY (destino_id) REFERENCES lugares(id)
          )
      ''')

      comitar_bd(conn)
      fechar_conexao_bd(conn)
      return True
    return False

def listar_lugares_bd():
    """
    Consulta a tabela 'lugares' e retorna uma lista de dicionários.
    As chaves dos dicionários são os nomes dos campos da tabela.
    """
    conn = abrir_conexao_bd()
    if conn:
        cursor = conn.cursor()
        cursor.execute('''
          SELECT id,
            descricao,
            endereco,
            numero,
            bairro,
            cep,
            cidade,
            uf,
            finalidade,
            data_cadastro,
            hora_cadastro
          FROM
            lugares
          ORDER BY
            descricao
        ''')
        registros = cursor.fetchall()
        fechar_conexao_bd(conn)

        colunas = ["id", "descricao", "endereco", "numero", "bairro", "cep", "cidade", "uf", "finalidade", "data_cadastro", "hora_cadastro"]
        lugares_dict = []
        for linha in registros:
            lugar = {}
            for i, coluna in enumerate(colunas):
                valor = linha[i]
                if coluna == "data_cadastro":
                    lugar[coluna] = formata_data(valor)
                elif coluna == "hora_cadastro":
                    lugar[coluna] = formata_hora(valor)
                else:
                    lugar[coluna] = valor
            lugares_dict.append(lugar)
        return lugares_dict
    else:
        return []

def listar_lugares_id_bd(id):
    """
    Consulta a tabela 'lugares' filtrando o id e retorna um dicionário com os dados do lugar.
    Retorna None se o id não for encontrado.
    """
    conn = abrir_conexao_bd()
    if conn:
        cursor = conn.cursor()
        cursor.execute('''
            SELECT
                id,
                descricao,
                endereco,
                numero,
                bairro,
                cep,
                cidade,
                uf,
                finalidade,
                data_cadastro,
                hora_cadastro
            FROM
                lugares
            WHERE
                id = ?
        ''', (id,))
        registro = cursor.fetchone()
        fechar_conexao_bd(conn)

        if registro:
            colunas = ["id", "descricao", "endereco", "numero", "bairro", "cep", "cidade", "uf", "finalidade", "data_cadastro", "hora_cadastro"]
            lugar = {}
            for i, coluna in enumerate(colunas):
                valor = registro[i]
                if coluna == "data_cadastro":
                    lugar[coluna] = formata_data(valor)
                elif coluna == "hora_cadastro":
                    lugar[coluna] = formata_hora(valor)
                else:
                    lugar[coluna] = valor
            return lugar
        else:
            return []
    else:
        return []

def listar_lugares_finalidade_bd(finalidade):
    """
    Consulta a tabela 'lugares' filtrando a finalidade e retorna um dicionário com os dados do lugar.
    Retorna None se o id não for encontrado.
    """
    conn = abrir_conexao_bd()
    if conn:
        cursor = conn.cursor()
        cursor.execute('''
        SELECT
          id,
          descricao,
          endereco,
          numero,
          bairro,
          cep,
          cidade,
          uf,
          finalidade,
          data_cadastro,
          hora_cadastro
        FROM
          lugares
          WHERE finalidade = ?
        ORDER BY descricao
        ''', (finalidade,))
        registros = cursor.fetchall()
        fechar_conexao_bd(conn)

        colunas = ["id", "descricao", "endereco", "numero", "bairro", "cep", "cidade", "uf", "finalidade", "data_cadastro", "hora_cadastro"]
        lugares_dict = []
        for linha in registros:
            lugar = {}
            for i, coluna in enumerate(colunas):
                valor = linha[i]
                if coluna == "data_cadastro":
                    lugar[coluna] = formata_data(valor)
                elif coluna == "hora_cadastro":
                    lugar[coluna] = formata_hora(valor)
                else:
                    lugar[coluna] = valor
            lugares_dict.append(lugar)
        return lugares_dict
    else:
        return []

def listar_configuracoes_bd():
    conn = abrir_conexao_bd()
    if conn:
        cursor = conn.cursor()
        cursor.execute('''
            SELECT
                conf.id,
                (lgo.descricao || ", " || lgo.cidade || " - " || lgo.uf) AS origem,
                (lgd.descricao || ", " || lgd.cidade || " - " || lgd.uf) AS destino,
                conf.preco_medio_uber_veiculo_km,
                conf.data_cadastro,
                conf.hora_cadastro
            FROM
                configuracoes AS conf
                INNER JOIN lugares AS lgo ON conf.origem_id = lgo.id
                INNER JOIN lugares AS lgd ON conf.destino_id = lgd.id
            LIMIT 1
        ''')
        registros = cursor.fetchall()
        fechar_conexao_bd(conn)
        colunas = ["id", "origem", "destino", "preco_medio_uber_veiculo_km", "data_cadastro", "hora_cadastro"]
        configuracoes_dict = []
        for linha in registros:
            lugar = {}
            for i, coluna in enumerate(colunas):
                valor = linha[i]
                if coluna == "data_cadastro":
                    lugar[coluna] = formata_data(valor)
                elif coluna == "hora_cadastro":
                    lugar[coluna] = formata_hora(valor)
                else:
                    lugar[coluna] = valor
            configuracoes_dict.append(lugar)
        return configuracoes_dict
    else:
        return []

def listar_rotas_bd():
    conn = abrir_conexao_bd()
    if conn:
        cursor = conn.cursor()
        cursor.execute('''
            SELECT
                rot.id,
                (lgo.descricao || ", " || lgo.cidade || " - " || lgo.uf) AS origem,
                (lgd.descricao || ", " || lgd.cidade || " - " || lgd.uf) AS destino,
                rot.quantidade_km,
                rot.link_rota,
                rot.data_cadastro,
                rot.hora_cadastro
            FROM
                rotas AS rot
                INNER JOIN lugares AS lgo ON rot.origem_id = lgo.id
                INNER JOIN lugares AS lgd ON rot.destino_id = lgd.id
                ORDER BY
                origem,
                destino
        ''')
        registros = cursor.fetchall()
        fechar_conexao_bd(conn)
        colunas = ["id", "origem", "destino", "quantidade_km", "link_rota", "data_cadastro", "hora_cadastro"]
        rotas_dict = []
        for linha in registros:
            lugar = {}
            for i, coluna in enumerate(colunas):
                valor = linha[i]
                if coluna == "data_cadastro":
                    lugar[coluna] = formata_data(valor)
                elif coluna == "hora_cadastro":
                    lugar[coluna] = formata_hora(valor)
                else:
                    lugar[coluna] = valor
            rotas_dict.append(lugar)
        return rotas_dict
    else:
        return []

def retorna_valor_total_km_rota_bd():
    conn = abrir_conexao_bd()
    if conn:
        cursor = conn.cursor()
        cursor.execute('''
            SELECT
                SUM(quantidade_km) AS valor_total_km_rota
            FROM
                rotas
            ''')
        registros = cursor.fetchall()
        fechar_conexao_bd(conn)
        return registros[0][0]
    else:
        return 0

def retorna_valor_uber_veiculo_configuracoes():
    configuracoes = listar_configuracoes_bd()
    if configuracoes:
        return configuracoes[0]['preco_medio_uber_veiculo_km']
    return None

def retorna_valor_total_estimado_gasto():
    valor_total_km_rota = retorna_valor_total_km_rota_bd()
    valor_uber_veiculo = retorna_valor_uber_veiculo_configuracoes()
    if valor_total_km_rota and valor_uber_veiculo:
        return valor_total_km_rota * valor_uber_veiculo
    return 0

def cadastrar_lugar_bd(campos, funcao_chamadora=1):
    conn = abrir_conexao_bd()
    if conn:
      finalidade = campos['finalidade']
      if funcao_chamadora == 1:
          if finalidade == 'O' or finalidade == 'D':
              if listar_lugares_finalidade_bd(campos['finalidade']):
                  if finalidade == 'O':
                      print("Origem já cadastrado")
                  else:
                      print("Destino já cadastrado")
                  return False

      cursor = conn.cursor()
      cursor.execute('''
          INSERT INTO lugares (
              descricao,
              endereco,
              numero,
              bairro,
              cep,
              cidade,
              uf,
              finalidade
          ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
          ''', (
          campos['descricao'],
          campos['endereco'],
          campos['numero'],
          campos['cep'],
          campos['bairro'],
          campos['cidade'],
          campos['uf'],
          finalidade
      ))
      comitar_bd(conn)
      fechar_conexao_bd(conn)
      return True

    return False

def cadastrar_rota_bd(campos):
    conn = abrir_conexao_bd()
    if conn:
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO rotas (
                origem_id,
                destino_id,
                quantidade_km,
                link_rota
            ) VALUES (?, ?, ?, ?)
            ''', (
            campos['origem_id'],
            campos['destino_id'],
            campos['quantidade_km'],
            campos['link_rota']
        ))
        comitar_bd(conn)
        fechar_conexao_bd(conn)
        return True
    return False

def cadastrar_configuracao_bd(campos):
    conn = abrir_conexao_bd()
    if conn:
        cursor = conn.cursor()
        configuracao = listar_configuracoes_bd()
        if configuracao:
            cursor.execute('''
                UPDATE configuracoes
                SET
                    origem_id = ?,
                    destino_id = ?,
                    preco_medio_uber_veiculo_km = ?
                WHERE id = ?
            ''', (
                campos['origem_id'],
                campos['destino_id'],
                campos['preco_medio_uber_veiculo_km'],
                configuracao[0]['id']
            ))
        else:
            cursor.execute('''
                INSERT INTO configuracoes (
                    origem_id,
                    destino_id,
                    preco_medio_uber_veiculo_km
                ) VALUES (?, ?, ?)
                ''', (
                campos['origem_id'],
                campos['destino_id'],
                campos['preco_medio_uber_veiculo_km']
            ))
        comitar_bd(conn)
        fechar_conexao_bd(conn)
        return True
    return False

def alterar_lugar_bd(id, campos):
    conn = abrir_conexao_bd()
    if conn:
        cursor = conn.cursor()
        cursor.execute('''
            UPDATE lugares
            SET
                descricao = ?,
                endereco = ?,
                numero = ?,
                bairro = ?,
                cep = ?,
                cidade = ?,
                uf = ?,
                finalidade = ?
            WHERE id = ?
            ''', (
            campos['descricao'],
            campos['endereco'],
            campos['numero'],
            campos['bairro'],
            campos['cep'],
            campos['cidade'],
            campos['uf'],
            campos['finalidade'],
            id
        ))
        comitar_bd(conn)
        fechar_conexao_bd(conn)
        return True
    return False

def alterar_configuracao_bd(id, campos):
    conn = abrir_conexao_bd()
    if conn:
        cursor = conn.cursor()
        cursor.execute('''
            UPDATE configuracoes
            SET
                origem_id = ?,
                destino_id = ?,
                preco_medio_uber_veiculo_km = ?
            WHERE id = ?
            ''', (
            campos['origem_id'],
            campos['destino_id'],
            campos['preco_medio_uber_veiculo_km'],
            id
        ))
        comitar_bd(conn)
        fechar_conexao_bd(conn)
        return True
    return False

def exluir_lugar_bd(id):
    conn = abrir_conexao_bd()
    if conn:
        cursor = conn.cursor()
        cursor.execute('''
            DELETE FROM lugares
            WHERE id = ?
            ''', (id,))
        comitar_bd(conn)
        fechar_conexao_bd(conn)
        return True
    return False

def retorna_endereco_formatado(lugar_obj): # Alterado o nome do parâmetro para clareza
    """
    Formata o endereço de um lugar (objeto dicionário).
    """
    if not lugar_obj: # Verifica se o objeto lugar não é None ou vazio
        return "Endereço indisponível"

    endereco = lugar_obj.get("endereco")
    numero = lugar_obj.get("numero")
    bairro = lugar_obj.get("bairro")
    cep = lugar_obj.get("cep")
    descricao = lugar_obj.get("descricao", "Descrição indisponível") # Valor padrão
    cidade = lugar_obj.get("cidade", "Cidade indisponível") # Valor padrão
    uf = lugar_obj.get("uf", "UF indisponível") # Valor padrão

    # Verifica se os campos essenciais para o endereço completo existem e não são vazios/None
    # Considera número como string para tratar casos onde pode ser None ou vazio
    if not endereco or not str(numero if numero is not None else '').strip() or not bairro or not cep:
        return f"{descricao}, {cidade} - {uf}"
    else:
        return f"{endereco}, {numero} - {bairro}, {cidade} - {uf}, {cep}"

# **4.1 - Teste de funções**

In [12]:
# criar_banco_tabelas()

# campos_lugares = {}
# campos_lugares['descricao'] = 'Praça do tomba'
# campos_lugares['endereco'] = ''
# campos_lugares['numero'] = None
# campos_lugares['bairro'] = None
# campos_lugares['cep'] = None
# campos_lugares['cidade'] = 'Feira de Santana'
# campos_lugares['uf'] = 'BA'
# campos_lugares['finalidade'] = 'O'

# cadastrar_lugar_bd(campos_lugares)

# campos_lugares = {}
# campos_lugares['descricao'] = 'Praia do meio'
# campos_lugares['endereco'] = ''
# campos_lugares['numero'] = None
# campos_lugares['bairro'] = None
# campos_lugares['cep'] = None
# campos_lugares['cidade'] = 'Aracaju'
# campos_lugares['uf'] = 'SE'
# campos_lugares['finalidade'] = 'D'

# cadastrar_lugar_bd(campos_lugares)

# campos_lugares = {}
# campos_lugares['descricao'] = 'Praia de Itapuã'
# campos_lugares['endereco'] = ''
# campos_lugares['numero'] = None
# campos_lugares['bairro'] = Itapuã
# campos_lugares['cep'] = None
# campos_lugares['cidade'] = 'Salvador'
# campos_lugares['uf'] = 'BA'
# campos_lugares['finalidade'] = 'D'
# cadastrar_lugar_bd(campos_lugares)

# campos_lugares = {}
# campos_lugares['descricao'] = 'Mercado Municipal Antônio Franco'
# campos_lugares['endereco'] = 'Av. João Ribeiro'
# campos_lugares['numero'] = 350
# campos_lugares['bairro'] = 'Atalaia'
# campos_lugares['cep'] = '49060-330'
# campos_lugares['cidade'] = 'Aracaju'
# campos_lugares['uf'] = 'SE'
# campos_lugares['finalidade'] = 'V'

# cadastrar_lugar_bd(campos_lugares)

# campos_lugares = {}
# campos_lugares['descricao'] = 'Sal e Brasa'
# campos_lugares['endereco'] = 'Av. Santos Dumont'
# campos_lugares['numero'] = 75
# campos_lugares['bairro'] = 'Coroa do Meio'
# campos_lugares['cep'] = '49035-785'
# campos_lugares['cidade'] = 'Aracaju'
# campos_lugares['uf'] = 'SE'
# campos_lugares['finalidade'] = 'V'

# cadastrar_lugar_bd(campos_lugares)

# campos_lugares = {}
# campos_lugares['descricao'] = 'Croa do Coré'
# campos_lugares['endereco'] = 'Mosqueiro'
# campos_lugares['numero'] = None
# campos_lugares['bairro'] = 'Mosqueiro'
# campos_lugares['cep'] = None
# campos_lugares['cidade'] = 'Aracaju'
# campos_lugares['uf'] = 'SE'
# campos_lugares['finalidade'] = 'V'

# cadastrar_lugar_bd(campos_lugares)

# campos_lugares = {}
# campos_lugares['descricao'] = 'Pousada da Praia'
# campos_lugares['endereco'] = 'R. Prof. Jugurta Feitosa Franco'
# campos_lugares['numero'] = 235
# campos_lugares['bairro'] = 'Coroa do Meio'
# campos_lugares['cep'] = '49035-690'
# campos_lugares['cidade'] = 'Aracaju'
# campos_lugares['uf'] = 'SE'
# campos_lugares['finalidade'] = 'H'

# cadastrar_lugar_bd(campos_lugares)

# campos_configuracoes = {}
# campos_configuracoes['origem_id'] = 1
# campos_configuracoes['destino_id'] = 2
# campos_configuracoes['preco_medio_uber_veiculo_km'] = 2.50

# cadastrar_configuracao_bd(campos_configuracoes)

In [13]:
# lugares = listar_lugares_bd()
# print(f"Lugares: {lugares}")

# lugar = listar_lugares_filtro_bd(1)
# print(f"Lugar: {lugar}")

# lugar_origem = listar_lugares_finalidade_bd('O')
# print(f"Lugar Origem: {lugar_origem}")

# lugar_destino = listar_lugares_finalidade_bd('D')
# print(f"Lugar Destino: {lugar_destino}")

# lugar_visita = listar_lugares_finalidade_bd('V')
# print(f"Lugar Visita: {lugar_visita}")

# lugar_hospedagem = listar_lugares_finalidade_bd('H')
# print(f"Lugar Hospedagem: {lugar_hospedagem}")



# **5 - Funções cliente para listar, cadastrar, alterar e excluir**

In [14]:
linha_separadora = "-" * 100
espacos = " " * 8

def listar_lugar_origem_cliente():

    """
    Função cliente para listar um lugar de origem no banco de dados.
    """

    print(f"{espacos}{linha_separadora}\n{espacos}Lugar de origem\n{espacos}{linha_separadora}")
    lugar = listar_lugares_finalidade_bd('O')
    if lugar:
        id = str(lugar[0]['id'])
        descricao = lugar[0]['descricao']
        endereco = f"{lugar[0]['endereco']}, {str(lugar[0]['numero'])} - {lugar[0]['bairro']}, {lugar[0]['cidade']} - {lugar[0]['uf']}, {lugar[0]['cep']}"

        print(f'''
        {linha_separadora}
        id: {id}
        Descrição: {descricao}
        Endereço: {endereco}.
        ''')
        return
    print("Aviso: Não há lugar de origem cadastrado.")

def listar_lugar_destino_cliente():
    """
    Função cliente para listar um lugar de destino no banco de dados.
    """
    print(f"{espacos}{linha_separadora}\n{espacos}Lugar de destino\n{espacos}{linha_separadora}")

    lugar = listar_lugares_finalidade_bd('D')
    if lugar:
        id = str(lugar[0]['id'])
        descricao = f"{lugar[0]['descricao']}, {str(lugar[0]['cidade'])} - {lugar[0]['uf']}."

        print(f'''
        {linha_separadora}
        id: {id}
        Descrição: {descricao}
        ''')
        return
    print("Aviso: Não há lugar de destino cadastrado.")

def listar_lugar_visita_cliente():
    """
    Função cliente para listar um lugar de visita no banco de dados.
    """
    print(f"{espacos}{linha_separadora}\n{espacos}Lugares de visita\n{espacos}{linha_separadora}")

    lugares = listar_lugares_finalidade_bd('V')
    if lugares:
        for i in range(len(lugares)):
            id = str(lugares[i]['id'])
            descricao = lugares[i]['descricao']
            endereco = f"{lugares[i]['endereco']}, {str(lugares[i]['numero'])} - {lugares[i]['bairro']}, {lugares[i]['cidade']} - {lugares[i]['uf']}, {lugares[i]['cep']}"

            print(f'''
            {linha_separadora}
            id: {id}
            Descrição: {descricao}
            Endereço: {endereco}
            ''')
        return
    print("Aviso: Não há lugares de visita cadastrados.")

def listar_lugar_hospedagem_cliente():
    """
    Função cliente para listar um lugar de hospedagem no banco de dados.
    """
    print(f"{espacos}{linha_separadora}\n{espacos}Lugar de hospedagem\n{espacos}{linha_separadora}")

    lugar = listar_lugares_finalidade_bd('H')
    if lugar:
        id = str(lugar[0]['id'])
        descricao = lugar[0]['descricao']
        endereco = f"{lugar[0]['endereco']}, {str(lugar[0]['numero'])} - {lugar[0]['bairro']}, {lugar[0]['cidade']} - {lugar[0]['uf']}, {lugar[0]['cep']}"

        print(f'''
        {linha_separadora}
        id: {id}
        Descrição: {descricao}
        Endereço: {endereco}
        ''')
        return
    print("Aviso: Não há lugar de hospedagem cadastrado.")

def listar_configuracoes_cliente():
    """
    Função cliente para listar as configurações no banco de dados.
    """
    print(f"{espacos}{linha_separadora}\n{espacos}Configurações do sistema\n{espacos}{linha_separadora}")

    configuracoes = listar_configuracoes_bd()
    if configuracoes:
        origem = configuracoes[0]['origem']
        destino = configuracoes[0]['destino']
        preco_medio_uber_veiculo_km = configuracoes[0]['preco_medio_uber_veiculo_km']

        print(f'''
        {linha_separadora}
        Origem: {origem}
        Destino: {destino}
        Preço médio do Uber por km: {preco_medio_uber_veiculo_km}
        ''')
    print("Aviso: O sistema não está parametrizado.")

def listar_rotas_cliente():
    """
    Função cliente para listar as rotas no banco de dados.
    """
    print(f"{espacos}{linha_separadora}\n{espacos}Rotas cadastradas\n{espacos}{linha_separadora}")

    preco_medio_uber_veiculo = retorna_valor_uber_veiculo_configuracoes()
    rotas = listar_rotas_bd()
    if rotas:
        for i in range(len(rotas)):
            id = str(rotas[i]['id'])
            origem = rotas[i]['origem']
            destino = rotas[i]['destino']
            quantidade_km = rotas[i]['quantidade_km']
            link_rota = rotas[i]['link_rota']
            data_cadastro = rotas[i]['data_cadastro']
            hora_cadastro = rotas[i]['hora_cadastro']
            valor_estimado = quantidade_km * preco_medio_uber_veiculo
            print(f'''
            {linha_separadora}
          id: {id}
          Origem: {origem}
          Destino: {destino}
          Quantidade de km: {str(formata_numero(quantidade_km))} Km
          Preço médio do Uber ou veículo por km: R$ {str(formata_numero(preco_medio_uber_veiculo))}
          Valor estimado: R$ {str(formata_numero(valor_estimado))}
          Link da rota: {link_rota}
          Data de cadastro: {data_cadastro}
          Hora de cadastro: {hora_cadastro}
            ''')
            return
    print("Aviso: Não há rotas cadastrada.")

def cadastrar_lugar_origem_cliente():
    """
    Função cliente para cadastrar um novo lugar de origem no banco de dados com menu interativo.
    """
    print(f"{espacos}{linha_separadora}\n{espacos}Cadastro de Novo Lugar de Origem\n{espacos}{linha_separadora}")

    lugar_orogigem = listar_lugares_finalidade_bd('O')
    if not lugar_orogigem:
        finalidade = 'O'
        campos = retorna_campos_cadastro_lugar()
        campos['finalidade'] = finalidade

        if cadastrar_lugar_bd(campos):
            print("\nLugar de origem cadastrado com sucesso!")
            return
        print("\nErro ao cadastrar o lugar.")
        return
    else:
        print("\nAviso: Já existe um lugar de origem cadastrado.")
        return

def cadastrar_lugar_destino_cliente():
    """
    Função cliente para cadastrar um novo lugar de destino no banco de dados com menu interativo.
    """
    print(f"{espacos}{linha_separadora}\n{espacos}Cadastro de Novo Lugar de Destino\n{espacos}{linha_separadora}")
    lugar_destino = listar_lugares_finalidade_bd('D')
    if not lugar_destino:
        finalidade = 'D'
        campos = retorna_campos_cadastro_lugar()
        campos['finalidade'] = finalidade

        if cadastrar_lugar_bd(campos):
            print("\nLugar de destino cadastrado com sucesso!")
            return
        print("\nErro ao cadastrar o lugar.")
        return
    else:
        print("\nAviso: Já existe um lugar de destino cadastrado.")

def cadastrar_lugar_hospedagem_cliente():
    """
    Função cliente para cadastrar um novo lugar de hospedagem no banco de dados com menu interativo.
    """
    print(f"{espacos}{linha_separadora}\n{espacos}Cadastro de Novo Lugar de Hospedagem\n{espacos}{linha_separadora}")
    lugar_hospedagem = listar_lugares_finalidade_bd('H')
    if not lugar_hospedagem:
        finalidade = 'H'
        campos = retorna_campos_cadastro_lugar()
        campos['finalidade'] = finalidade

        if cadastrar_lugar_bd(campos):
            print("\nLugar de hospedagem cadastrado com sucesso!")
            return
        print("\nErro ao cadastrar o lugar.")
        return
    else:
        print("\nAviso: Já existe um lugar de hospedagem cadastrado.")
        return

def cadastrar_lugar_visita_cliente():
    """
    Função cliente para cadastrar um novo lugar de visita no banco de dados com menu interativo.
    """
    print(f"{espacos}{linha_separadora}\n{espacos}Cadastro de Novo Lugar de Visita\n{espacos}{linha_separadora}")

    lugar_visita = listar_lugares_finalidade_bd('V')

    if not lugar_visita:
        finalidade = 'V'
        campos = retorna_campos_cadastro_lugar()
        campos['finalidade'] = finalidade

        if cadastrar_lugar_bd(campos):
            print("\nLugar de visita cadastrado com sucesso!")
            return
        print("\nErro ao cadastrar o lugar.")
        return
    else:
        print("\nAviso: Já existe um lugar de visita cadastrado.")
        return

def cadastrar_lugar_cliente():
    """
    Função cliente para cadastrar um novo lugar no banco de dados com menu interativo.
    """
    print(f"{espacos}{linha_separadora}\n{espacos}Cadastro de Novo Lugar\n{espacos}{linha_separadora}")

    while True:
        print("\n Informe o tipo de finalidade:")
        print("1 - Origem. Lugar de onde você irá partir")
        print("2 - Destino. Para onde você pretende ir")
        print("3 - Hospedagem. Onde você pretende ficar hospedado.")
        print("4 - Visita. Lugar que você pretende visitar no seu destino.")
        print("5 - Voltar para o menu principal.")

        opcao = input("Selecione uma opção: ")
        # print(linha_separadora)
        if opcao == '1':
            if listar_lugares_finalidade_bd('O'):
                print("\nVocê já possui um lugar de origem cadastrado.")
                return
            else:
                finalidade = 'O'
                break
        elif opcao == '2':
            if listar_lugares_finalidade_bd('D'):
                print("\nVocê já possui um lugar de destino cadastrado.")
                return
            else:
                finalidade = 'D'
                break
        elif opcao == '3':
            if listar_lugares_finalidade_bd('H'):
                print("\nVocê já possui um lugar de hospedagem cadastrado.")
                return
            else:
                finalidade = 'H'
                break
        elif opcao == '4':
            finalidade = 'V'
            break
        elif opcao == '5':
            print("Voltando para o menu principal.")
            return
        else:
            print("Opção inválida. Por favor, selecione uma das opções do menu.")

    campos = retorna_campos_cadastro_lugar()
    campos['finalidade'] = finalidade

    if cadastrar_lugar_bd(campos):
        print("\nLugar cadastrado com sucesso!")
        return
    else:
        print("\nErro ao cadastrar o lugar.")
        return

def cadastrar_rota_cliente(origem_id, destino_id, quantidade_km, link_rota):
    """
    Função cliente para cadastrar uma nova rota no banco de dados.
    """
    try:
        campos = {
            'origem_id': origem_id,
            'destino_id': destino_id,
            'quantidade_km': quantidade_km,
            'link_rota': link_rota
        }

        if cadastrar_rota_bd(campos):
            print("\nRota cadastrada com sucesso!")
            return
        else:
            print("\nErro ao cadastrar a rota.")
            return

    except Exception as e:
        print(f"\nOcorreu um erro ao cadastrar a rota: {e}")
        return

def cadastrar_configuracao_cliente():
    """
    Função cliente para cadastrar uma nova configuração no banco de dados,
    buscando automaticamente os IDs da origem e do destino (se existirem).
    """
    print(f"{espacos}{linha_separadora}\n{espacos}Cadastro de Nova Configuração\n{espacos}{linha_separadora}")

    try:
        # 1 - Origem
        origens = listar_lugares_finalidade_bd('O')
        if not origens:
            print("Aviso: Não há lugar de origem cadastrado. Cadastre um primeiro para configurar.")
            return
        origem_id = origens[0]['id']

        # 2 - Destino
        destinos = listar_lugares_finalidade_bd('D')
        if not destinos:
            print("Aviso: Não há lugar de destino cadastrado. Cadastre um primeiro para configurar.")
            return
        destino_id = destinos[0]['id']

        # 3 - Preço médio do Uber ou veículo por km
        preco_str = input("Informe o preço médio do Uber ou veículo por km (ou 'S' para sair): ").upper()
        if preco_str == 'S':
            print("Operação cancelada.")
            return
        try:
            preco_medio_uber_veiculo_km = float(preco_str.replace(',', '.')) # Permite vírgula como separador decimal
        except ValueError:
            print("Erro: Preço inválido. Digite um número.")
            return

        campos = {
            'origem_id': origem_id,
            'destino_id': destino_id,
            'preco_medio_uber_veiculo_km': preco_medio_uber_veiculo_km
        }

        if cadastrar_configuracao_bd(campos):
            print("\nSistema parametrizado com sucesso!")
        else:
            print("\nErro ao cadastrar a configuração.")

    except Exception as e:
        print(f"\nOcorreu um erro ao cadastrar a configuração: {e}")

def exibe_gastos_totais_cliente():
    """
    Função cliente para exibir os gastos totais no banco de dados.
    """
    print(f"{espacos}{linha_separadora}\n{espacos}Gastos totais estimados de uber/veículo\n{espacos}{linha_separadora}")

    print(f"{espacos}O valor total estimado que você irá gastar de uber/veículo é de R$ {str(formata_numero(retorna_valor_total_estimado_gasto()))}")
    return


# **6 - Alterar lugar**

In [15]:
espaco = " " * 8
def alterar_lugar_cliente():
    """
    Função cliente para alterar um lugar no banco de dados com menu interativo.
    """
    print(f"{espaco}{linha_separadora}\n{espaco}Alterar Lugar\n{espaco}{linha_separadora}")

    while True:
        print(f"\nInforme o tipo de finalidade do lugar que deseja alterar:")
        print(f"1 - Origem. Lugar de onde você irá partir")
        print(f"2 - Destino. Para onde você pretende ir")
        print(f"3 - Hospedagem. Onde você pretende ficar hospedado.")
        print(f"4 - Visita. Lugar que você pretende visitar no seu destino.")
        print(f"5 - Voltar para o menu principal.")

        opcao = input("Selecione uma opção: ")

        if opcao == '1':
            lugares_origem = listar_lugares_finalidade_bd('O')

            if not lugares_origem:
                print("\nAviso: Não há lugar de origem cadastrado para alterar.")
            else:
                origem_antiga = lugares_origem[0]
                campos_antigo_origem = {
                    'descricao': origem_antiga['descricao'],
                    'endereco': origem_antiga['endereco'],
                    'numero': origem_antiga['numero'],
                    'bairro': origem_antiga['bairro'],
                    'cep': origem_antiga['cep'],
                    'cidade': origem_antiga['cidade'],
                    'uf': origem_antiga['uf'],
                    'finalidade': 'V'
                }
                print("\nJá existe um lugar de origem cadastrado:")
                listar_lugar_origem_cliente()
                escolha = input("Deseja selecionar um lugar de visita para se tornar a nova origem (S/N)? ").upper()
                if escolha == 'S':
                    lugares_visita = listar_lugares_finalidade_bd('V')
                    if not lugares_visita:
                        print("\nAviso: Não há lugares de visita cadastrados para selecionar.")
                    else:
                        print("\nSelecione o lugar de visita para ser a nova origem:")
                        for i, lugar in enumerate(lugares_visita):
                            print(f"{i + 1} - {lugar['descricao']}, {lugar['cidade']} - {lugar['uf']}")
                        while True:
                            opcao_visita_str = input("Digite o número do lugar de visita (ou 'S' para sair): ").upper()
                            if opcao_visita_str == 'S':
                                print("Operação cancelada.")
                                break
                            try:
                                opcao_visita = int(opcao_visita_str)
                                if 1 <= opcao_visita <= len(lugares_visita):
                                    origem_nova_id = lugares_visita[opcao_visita - 1]['id']
                                    if alterar_lugar_bd(origem_antiga['id'], campos_antigo_origem):
                                        lugar_novo_origem = lugares_visita[opcao_visita - 1]
                                        campos_novo_origem = {
                                            'descricao': lugar_novo_origem['descricao'],
                                            'endereco': lugar_novo_origem['endereco'],
                                            'numero': lugar_novo_origem['numero'],
                                            'bairro': lugar_novo_origem['bairro'],
                                            'cep': lugar_novo_origem['cep'],
                                            'cidade': lugar_novo_origem['cidade'],
                                            'uf': lugar_novo_origem['uf'],
                                            'finalidade': 'O'
                                        }
                                        if alterar_lugar_bd(origem_nova_id, campos_novo_origem):
                                            print("\nLugar de origem alterado com sucesso!")
                                        else:
                                            print("\nErro ao alterar o novo lugar para origem.")
                                    else:
                                        print("\nErro ao alterar o lugar de origem antigo para visita.")
                                    break
                                else:
                                    print("Opção inválida. Digite um número da lista.")
                            except ValueError:
                                print("Entrada inválida. Digite um número.")
                elif escolha == 'N':
                    if alterar_lugar_bd(origem_antiga['id'], campos_antigo_origem):
                        campos = retorna_campos_cadastro_lugar()
                        campos['finalidade'] = 'O'
                        if cadastrar_lugar_bd(campos, 2):
                            print("\n Novo lugar de origem cadastrado com sucesso!")
                            break
                        print("\nErro ao cadastrar o novo lugar de origem.")
                        break
                    else:
                        print("\nErro ao alterar o lugar de origem antigo para visita.")
                    break
                else:
                    print("Opção inválida.")
            break

        elif opcao == '2':
            lugares_destino = listar_lugares_finalidade_bd('D')
            if not lugares_destino:
                print("\nAviso: Não há lugar de destino cadastrado para alterar.")
            else:
                destino_antigo = lugares_destino[0]
                campos_antigo_destino = {
                    'descricao': destino_antigo['descricao'],
                    'endereco': destino_antigo['endereco'],
                    'numero': destino_antigo['numero'],
                    'bairro': destino_antigo['bairro'],
                    'cep': destino_antigo['cep'],
                    'cidade': destino_antigo['cidade'],
                    'uf': destino_antigo['uf'],
                    'finalidade': 'V'
                }
                escolha = input("Deseja selecionar um lugar de visita para se tornar o novo destino (S/N)? ").upper()
                if escolha == 'S':
                    lugares_visita = listar_lugares_finalidade_bd('V')
                    if not lugares_visita:
                        print("\nAviso: Não há lugares de visita cadastrados para selecionar.")
                    else:
                        print("\nSelecione o lugar de visita para ser o novo destino:")
                        for i, lugar in enumerate(lugares_visita):
                            print(f"{i + 1} - {lugar['descricao']}, {lugar['cidade']} - {lugar['uf']}")
                        while True:
                            opcao_visita_str = input("Digite o número do lugar de visita (ou 'S' para sair): ").upper()
                            if opcao_visita_str == 'S':
                                print("Operação cancelada.")
                                break
                            try:
                                opcao_visita = int(opcao_visita_str)
                                if 1 <= opcao_visita <= len(lugares_visita):
                                    destino_novo_id = lugares_visita[opcao_visita - 1]['id']
                                    if alterar_lugar_bd(destino_antigo['id'], campos_antigo_destino):
                                        lugar_novo_destino = lugares_visita[opcao_visita - 1]
                                        campos_novo_destino = {
                                            'descricao': lugar_novo_destino['descricao'],
                                            'endereco': lugar_novo_destino['endereco'],
                                            'numero': lugar_novo_destino['numero'],
                                            'bairro': lugar_novo_destino['bairro'],
                                            'cep': lugar_novo_destino['cep'],
                                            'cidade': lugar_novo_destino['cidade'],
                                            'uf': lugar_novo_destino['uf'],
                                            'finalidade': 'D'
                                        }
                                        if alterar_lugar_bd(destino_novo_id, campos_novo_destino):
                                            print("\nLugar de destino alterado com sucesso!")
                                        else:
                                            print("\nErro ao alterar o novo lugar para destino.")
                                    else:
                                        print("\nErro ao alterar o lugar de destino antigo para visita.")
                                    break
                                else:
                                    print("Opção inválida. Digite um número da lista.")
                            except ValueError:
                                print("Entrada inválida. Digite um número.")
                elif escolha == 'N':
                    if alterar_lugar_bd(destino_antigo['id'], campos_antigo_destino):
                        campos = retorna_campos_cadastro_lugar()
                        campos['finalidade'] = 'D'
                        cadastrar_lugar_bd(campos, 2)
                        print("\n Novo lugar de destino cadastrado com sucesso!")
                else:
                    print("Opção inválida.")
            break

        elif opcao == '3':
            lugares_hospedagem = listar_lugares_finalidade_bd('H')
            if not lugares_hospedagem:
                print("\nAviso: Não há lugar de hospedagem cadastrado para alterar.")
            else:
                hospedagem_antiga = lugares_hospedagem[0]
                campos_antigo_hospedagem = {
                    'descricao': hospedagem_antiga['descricao'],
                    'endereco': hospedagem_antiga['endereco'],
                    'numero': hospedagem_antiga['numero'],
                    'bairro': hospedagem_antiga['bairro'],
                    'cep': hospedagem_antiga['cep'],
                    'cidade': hospedagem_antiga['cidade'],
                    'uf': hospedagem_antiga['uf'],
                    'finalidade': 'V'
                }
                escolha = input("Deseja selecionar um lugar de visita para se tornar a nova hospedagem (S/N)? ").upper()
                if escolha == 'S':
                    lugares_visita = listar_lugares_finalidade_bd('V')
                    if not lugares_visita:
                        print("\nAviso: Não há lugares de visita cadastrados para selecionar.")
                    else:
                        print("\nSelecione o lugar de visita para ser a nova hospedagem:")
                        for i, lugar in enumerate(lugares_visita):
                            print(f"{i + 1} - {lugar['descricao']}, {lugar['cidade']} - {lugar['uf']}")
                        while True:
                            opcao_visita_str = input("Digite o número do lugar de visita (ou 'S' para sair): ").upper()
                            if opcao_visita_str == 'S':
                                print("Operação cancelada.")
                                break
                            try:
                                opcao_visita = int(opcao_visita_str)
                                if 1 <= opcao_visita <= len(lugares_visita):
                                    hospedagem_nova_id = lugares_visita[opcao_visita - 1]['id']
                                    if alterar_lugar_bd(hospedagem_antiga['id'], campos_antigo_hospedagem):
                                        lugar_novo_hospedagem = lugares_visita[opcao_visita - 1]
                                        campos_novo_hospedagem = {
                                            'descricao': lugar_novo_hospedagem['descricao'],
                                            'endereco': lugar_novo_hospedagem['endereco'],
                                            'numero': lugar_novo_hospedagem['numero'],
                                            'bairro': lugar_novo_hospedagem['bairro'],
                                            'cep': lugar_novo_hospedagem['cep'],
                                            'cidade': lugar_novo_hospedagem['cidade'],
                                            'uf': lugar_novo_hospedagem['uf'],
                                            'finalidade': 'H'
                                        }
                                        if alterar_lugar_bd(hospedagem_nova_id, campos_novo_hospedagem):
                                            print("\nLugar de hospedagem alterado com sucesso!")
                                        else:
                                            print("\nErro ao alterar o novo lugar para hospedagem.")
                                    else:
                                        print("\nErro ao alterar o lugar de hospedagem antigo para visita.")
                                    break
                                else:
                                    print("Opção inválida. Digite um número da lista.")
                            except ValueError:
                                print("Entrada inválida. Digite um número.")
                elif escolha == 'N':
                    if alterar_lugar_bd(hospedagem_antiga['id'], campos_antigo_hospedagem):
                        campos = retorna_campos_cadastro_lugar()
                        campos['finalidade'] = 'H'
                        cadastrar_lugar_bd(campos, 2)
                        print("\n Novo lugar de hospedagem cadastrada com sucesso!")
                else:
                    print("Opção inválida.")
            break

        elif opcao == '4':
            lugares_visita = listar_lugares_finalidade_bd('V')
            if not lugares_visita:
                print("\nAviso: Não há lugares de visita cadastrados para alterar.")
            else:
                print("\nSelecione o lugar de visita que deseja alterar:")
                for i, lugar in enumerate(lugares_visita):
                    print(f"{i + 1} - {lugar['descricao']}, {lugar['cidade']} - {lugar['uf']}")
                while True:
                    opcao_alterar_str = input("Digite o número do lugar de visita (ou 'S' para sair): ").upper()
                    if opcao_alterar_str == 'S':
                        print("Operação cancelada.")
                        break
                    try:
                        opcao_alterar = int(opcao_alterar_str)
                        if 1 <= opcao_alterar <= len(lugares_visita):
                            lugar_para_alterar = lugares_visita[opcao_alterar - 1]
                            print(f"\nInforme os novos dados para: {lugar_para_alterar['descricao']}")
                            descricao = input(f"Nova descrição ({lugar_para_alterar['descricao']}): ")
                            endereco = input(f"Novo endereço ({lugar_para_alterar['endereco']}): ")
                            numero = input(f"Novo número ({lugar_para_alterar['numero']}): ")
                            bairro = input(f"Novo bairro ({lugar_para_alterar['bairro']}): ")
                            cep = input(f"Novo CEP ({lugar_para_alterar['cep']}): ")
                            cidade = input(f"Nova cidade ({lugar_para_alterar['cidade']}): ")
                            uf = input(f"Nova UF ({lugar_para_alterar['uf']}): ").upper()

                            campos_alterados = {
                                'descricao': descricao,
                                'endereco': endereco,
                                'numero': numero,
                                'bairro': bairro,
                                'cep': cep,
                                'cidade': cidade,
                                'uf': uf,
                                'finalidade': 'V'
                            }
                            if alterar_lugar_bd(lugar_para_alterar['id'], campos_alterados):
                                print("\nLugar de visita alterado com sucesso!")
                            else:
                                print("\nErro ao alterar o lugar de visita.")
                            break
                        else:
                            print("Opção inválida. Digite um número da lista.")
                    except ValueError:
                        print("Entrada inválida. Digite um número.")
                break

        elif opcao == '5':
            print("Voltando para o menu principal.")
            return

        else:
            print("Opção inválida. Por favor, selecione uma das opções do menu.")

# alterar_lugar_cliente()

# **7 - Excluir lugares**


In [16]:
def excluir_lugar_cliente():
    """
    Função cliente para excluir um lugar do banco de dados com menu interativo.
    A listagem dos lugares segue o padrão "descricao, cidade - uf".
    """
    print(f"{espaco}{linha_separadora}\n{espaco}Excluir Lugar\n{espaco}{linha_separadora}")

    while True:
        lugares = listar_lugares_bd()
        if not lugares:
            print("\nAviso: Não há lugares cadastrados para excluir.")
            return

        print("\nSelecione o lugar que deseja excluir:")
        for i, lugar in enumerate(lugares):
            print(f"{i + 1} - {lugar['descricao']}, {lugar['cidade']} - {lugar['uf']} (ID: {lugar['id']}, Finalidade: {lugar['finalidade']})")

        opcao_excluir_str = input("Digite o número do lugar para excluir (ou '0' para sair): ").upper()
        if opcao_excluir_str == '0':
            print("Saindo do menu de exclusão de lugar.")
            return

        try:
            opcao_excluir = int(opcao_excluir_str)
            if 1 <= opcao_excluir <= len(lugares):
                lugar_para_excluir = lugares[opcao_excluir - 1]
                finalidade_excluir = lugar_para_excluir['finalidade']
                id_para_excluir = lugar_para_excluir['id']

                if finalidade_excluir == 'O':
                    print("\nVocê está excluindo um lugar de ORIGEM.")
                    confirmacao = input(f"Deseja realmente excluir '{lugar_para_excluir['descricao']}' (S/N)? ").upper()
                    if confirmacao == 'S':
                        lugares_visita = listar_lugares_finalidade_bd('V')
                        if lugares_visita:
                            print("\nSelecione um lugar de visita para se tornar a nova ORIGEM:")
                            for i, lugar_visita in enumerate(lugares_visita):
                                print(f"{i + 1} - {lugar_visita['descricao']}, {lugar_visita['cidade']} - {lugar_visita['uf']} (ID: {lugar_visita['id']})")
                            while True:
                                opcao_nova_origem_str = input("Digite o número do novo lugar de origem (ou 'S' para cancelar): ").upper()
                                if opcao_nova_origem_str == 'S':
                                    print("Operação de exclusão cancelada.")
                                    break
                                try:
                                    opcao_nova_origem = int(opcao_nova_origem_str)
                                    if 1 <= opcao_nova_origem <= len(lugares_visita):
                                        novo_origem = lugares_visita[opcao_nova_origem - 1]
                                        origem_antiga = lugar_para_excluir
                                        campos_novo_origem = {
                                            'descricao': novo_origem['descricao'],
                                            'endereco': novo_origem['endereco'],
                                            'numero': novo_origem['numero'],
                                            'bairro': novo_origem['bairro'],
                                            'cep': novo_origem['cep'],
                                            'cidade': novo_origem['cidade'],
                                            'uf': novo_origem['uf'],
                                            'finalidade': 'O'
                                        }
                                        if alterar_lugar_bd(novo_origem['id'], campos_novo_origem):
                                            if exluir_lugar_bd(id_para_excluir):
                                                print("\nLugar de origem excluído e um novo lugar de origem definido com sucesso!")
                                            else:
                                                print("\nErro ao excluir o lugar de origem antigo.")
                                        else:
                                            print("\nErro ao definir o novo lugar como origem.")
                                        return
                                    else:
                                        print("Opção inválida. Digite um número da lista.")
                                except ValueError:
                                    print("Entrada inválida. Digite um número.")
                            if opcao_nova_origem_str == 'S':
                                continue # Volta ao menu principal de exclusão
                        else:
                            print("\nAviso: Não há lugares de visita cadastrados para substituir a origem. Exclusão cancelada.")
                    else:
                        print("Exclusão cancelada.")

                elif finalidade_excluir == 'D':
                    print("\nVocê está excluindo um lugar de DESTINO.")
                    confirmacao = input(f"Deseja realmente excluir '{lugar_para_excluir['descricao']}' (S/N)? ").upper()
                    if confirmacao == 'S':
                        lugares_visita = listar_lugares_finalidade_bd('V')
                        if lugares_visita:
                            print("\nSelecione um lugar de visita para se tornar o novo DESTINO:")
                            for i, lugar_visita in enumerate(lugares_visita):
                                print(f"{i + 1} - {lugar_visita['descricao']}, {lugar_visita['cidade']} - {lugar_visita['uf']} (ID: {lugar_visita['id']})")
                            while True:
                                opcao_novo_destino_str = input("Digite o número do novo lugar de destino (ou 'S' para cancelar): ").upper()
                                if opcao_novo_destino_str == 'S':
                                    print("Operação de exclusão cancelada.")
                                    break
                                try:
                                    opcao_novo_destino = int(opcao_novo_destino_str)
                                    if 1 <= opcao_novo_destino <= len(lugares_visita):
                                        novo_destino = lugares_visita[opcao_novo_destino - 1]
                                        destino_antigo = lugar_para_excluir
                                        campos_novo_destino = {
                                            'descricao': novo_destino['descricao'],
                                            'endereco': novo_destino['endereco'],
                                            'numero': novo_destino['numero'],
                                            'bairro': novo_destino['bairro'],
                                            'cep': novo_destino['cep'],
                                            'cidade': novo_destino['cidade'],
                                            'uf': novo_destino['uf'],
                                            'finalidade': 'D'
                                        }
                                        if alterar_lugar_bd(novo_destino['id'], campos_novo_destino):
                                            if exluir_lugar_bd(id_para_excluir):
                                                print("\nLugar de destino excluído e um novo lugar de destino definido com sucesso!")
                                            else:
                                                print("\nErro ao excluir o lugar de destino antigo.")
                                        else:
                                            print("\nErro ao definir o novo lugar como destino.")
                                        return
                                    else:
                                        print("Opção inválida. Digite um número da lista.")
                                except ValueError:
                                    print("Entrada inválida. Digite um número.")
                            if opcao_novo_destino_str == 'S':
                                continue # Volta ao menu principal de exclusão
                        else:
                            print("\nAviso: Não há lugares de visita cadastrados para substituir o destino. Exclusão cancelada.")
                    else:
                        print("Exclusão cancelada.")

                elif finalidade_excluir == 'H':
                    print("\nVocê está excluindo um lugar de HOSPEDAGEM.")
                    confirmacao = input(f"Deseja realmente excluir '{lugar_para_excluir['descricao']}' (S/N)? ").upper()
                    if confirmacao == 'S':
                        lugares_visita = listar_lugares_finalidade_bd('V')
                        if lugares_visita:
                            print("\nSelecione um lugar de visita para se tornar a nova HOSPEDAGEM:")
                            for i, lugar_visita in enumerate(lugares_visita):
                                print(f"{i + 1} - {lugar_visita['descricao']}, {lugar_visita['cidade']} - {lugar_visita['uf']} (ID: {lugar_visita['id']})")
                            while True:
                                opcao_nova_hospedagem_str = input("Digite o número do novo lugar de hospedagem (ou 'S' para cancelar): ").upper()
                                if opcao_nova_hospedagem_str == 'S':
                                    print("Operação de exclusão cancelada.")
                                    break
                                try:
                                    opcao_nova_hospedagem = int(opcao_nova_hospedagem_str)
                                    if 1 <= opcao_nova_hospedagem <= len(lugares_visita):
                                        novo_hospedagem = lugares_visita[opcao_nova_hospedagem - 1]
                                        hospedagem_antiga = lugar_para_excluir
                                        campos_novo_hospedagem = {
                                            'descricao': novo_hospedagem['descricao'],
                                            'endereco': novo_hospedagem['endereco'],
                                            'numero': novo_hospedagem['numero'],
                                            'bairro': novo_hospedagem['bairro'],
                                            'cep': novo_hospedagem['cep'],
                                            'cidade': novo_hospedagem['cidade'],
                                            'uf': novo_hospedagem['uf'],
                                            'finalidade': 'H'
                                        }
                                        if alterar_lugar_bd(novo_hospedagem['id'], campos_novo_hospedagem):
                                            if exluir_lugar_bd(id_para_excluir):
                                                print("\nLugar de hospedagem excluído e um novo lugar de hospedagem definido com sucesso!")
                                            else:
                                                print("\nErro ao excluir o lugar de hospedagem antigo.")
                                        else:
                                            print("\nErro ao definir o novo lugar como hospedagem.")
                                        return
                                    else:
                                        print("Opção inválida. Digite um número da lista.")
                                except ValueError:
                                    print("Entrada inválida. Digite um número.")
                            if opcao_nova_hospedagem_str == 'S':
                                continue # Volta ao menu principal de exclusão
                        else:
                            print("\nAviso: Não há lugares de visita cadastrados para substituir a hospedagem. Exclusão cancelada.")
                    else:
                        print("Exclusão cancelada.")

                elif finalidade_excluir == 'V':
                    print("\nVocê está excluindo um lugar de VISITA.")
                    confirmacao = input(f"Deseja realmente excluir '{lugar_para_excluir['descricao']}' (S/N)? ").upper()
                    if confirmacao == 'S':
                        if exluir_lugar_bd(id_para_excluir):
                            print("\nLugar de visita excluído com sucesso!")
                        else:
                            print("\nErro ao excluir o lugar de visita.")
                    else:
                        print("Exclusão cancelada.")
                    break # Sai do loop de exclusão após tentar excluir uma visita

            else:
                print("Opção inválida. Digite um número da lista.")
        except ValueError:
            print("Entrada inválida. Digite um número.")

# **7.1 - Teste de funções cliente**

In [17]:
# listar_lugar_origem_cliente()
# listar_lugar_destino_cliente()
# listar_lugar_visita_cliente()
# listar_lugar_hospedagem_cliente()
# cadastrar_lugar_cliente()
# cadastrar_rota_cliente(1, 2, 321.23, 'https://www.google.com/maps/dir/Pra%C3%A7a+Mac%C3%A1rio+Barreto+-+Tomba,+Travessa+M%C3%A9xico,+2-52+-+Tomba,+Feira+de+Santana+-+BA/Praia+de+Atalaia,+Aracaju+-+SE/@-12.0351869,-39.1093011,8.25z/data=!4m13!4m12!1m5!1m1!1s0x714364d43745559:0xd32f83c84b0e6012!2m2!1d-38.9571467!2d-12.2840783!1m5!1m1!1s0x71ab128b11bbd55:0xedb902da2fa85146!2m2!1d-37.0365845!2d-10.9788554?entry=ttu&g_ep=EgoyMDI1MDUxMy4xIKXMDSoASAFQAw%3D%3D')
# cadastrar_configuracao_cliente()
# listar_configuracoes_cliente()

# excluir_lugar_cliente()

# listar_rotas_cliente()

# **8 - Agentes de IA**

In [18]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

**Agente buscador_rota**

In [19]:
##########################################
# --- Agente 1: Buscador de rota --- #
##########################################

def agente_buscador_rota(origem_endereco, destino_endereco):
  buscador = Agent(
      name="agente_buscador_rota",
      model="gemini-2.0-flash",
      instruction="""
        Você é um assistente de viagem para um projeto de desenvolvimento de software.
        Irei te passar dois parâmetros, origem e destino e você deverá retornar o melhor caminho de viagem entre eles usando o formato JSON pré estabelecido logo abaixo.

        Exemplos de parâmetros:
          origem_endereco : Rua tabapuã, 512 - Tomba, Feira de Santana - BA, 44090-520.
          destino_endereco: Praia de Atalaia, Aracaju - SE, 49000-000.
        Você terá as seguintes funcionalidades:

          1 - Irei passar a origem e destino como parâmetros
          e você buscará a melhor rota para o endereço de origem e endereço de destino.
          2 - Você deverá retornar os seguintes campos em JSON, seguindo os seguintes criterios:
            A - O endereço de origem (origem_endereco): Rua tabapuã, 268 - Tomba, Feira de Santana - BA, 44090-520
            B - O endereço de destino (destino_endereco): Praia de Atalaia, Aracaju - SE, 49000-000.
            C - A distância da viagem (distancia_viagem_km): 350.35. Sempre em formado decimal de até 2 casas.
            D - link da rota do google mapsno seguinte formato (link_rota): "https://www.google.com/maps/dir/R.+Tabapua,+268+-+Tomba,+Feira+de+Santana+-+BA,+44090-520/Praia+de+Atalaia,+Aracaju+-+SE/@-11.848997,-39.3178655,8z/data=!3m1!4b1!4m13!4m12!1m5!1m1!1s0x71437b39667809b:0x8e9dd3a09ee61879!2m2!1d-38.9604431!2d-12.2789361!1m5!1m1!1s0x71ab128b11bbd55:0xedb902da2fa85146!2m2!1d-37.0365845!2d-10.9788554?entry=ttu&g_ep=EgoyMDI1MDUxMy4xIKXMDSoASAFQAw%3D%3D"
            E - Descrição da rota (descricao_rota): "Via BR 101"
        Exemplo de de json esperado na resposta:
          {
            "origem_endereco": "Rua tabapuã, 268 - Tomba, Feira de Santana - BA, 44090-520",
            "destino_endereco": "Praia de Atalaia, Aracaju - SE, 49000-000",
            "distancia_viagem_km": 318.00,
            "link_rota": "https://www.google.com/maps/dir/R.+Tabapua,+268+-+Tomba,+Feira+de+Santana+-+BA,+44090-520/Praia+de+Atalaia,+Aracaju+-+SE/@-11.848997,-39.3178655,8z/data=!3m1!4b1!4m13!4m12!1m5!1m1!1s0x71437b39667809b:0x8e9dd3a09ee61879!2m2!1d-38.9604431!2d-12.2789361!1m5!1m1!1s0x71ab128b11bbd55:0xedb902da2fa85146!2m2!1d-37.0365845!2d-10.9788554?entry=ttu&g_ep=EgoyMDI1MDUxMy4xIKXMDSoASAFQAw%3D%3D",
            "descricao_rota": "Via BR 101"
          }
      """,
      description="Agente que busca a melhor rota entre dois endereços",
  )

  entrada_do_agente_buscador = f"Melhor rota entre {origem_endereco} e  {destino_endereco}."
  # Executa o agente
  rota = call_agent(buscador, entrada_do_agente_buscador)
  return rota

# **9 - Formatar respostas dos agentes**

In [20]:

def formata_resposta_agente_buscador_rota(origem_endereco, destino_endereco):
    # Chamada do agente
    resposta = agente_buscador_rota(origem_endereco, destino_endereco)

    # Remove marcações de bloco de código Markdown (```json e ```)
    resposta_limpa = re.sub(r"^```(?:json)?\s*|\s*```$", "", resposta.strip(), flags=re.IGNORECASE)

    # Converte para dicionário
    try:
        dados_rota = json.loads(resposta_limpa)
        # print("Dados convertidos com sucesso!")
        return dados_rota
    except json.JSONDecodeError as e:
        return f"Erro ao decodificar JSON:{e} \n Conteúdo recebido: {resposta_limpa}"

formata_resposta_agente_buscador_rota(
    'Rua tabapuã, 268 - Tomba, Feira de Santana - Ba, 44090-652',
    'Praia de Atalaia, Aracaju - SE')

{'origem_endereco': 'Rua Tabapuã, 268 - Tomba, Feira de Santana - BA, 44090-652',
 'destino_endereco': 'Praia de Atalaia, Aracaju - SE, Brasil',
 'distancia_viagem_km': 318.0,
 'link_rota': 'https://www.google.com/maps/dir/R.+Tabapu%C3%A3,+268+-+Tomba,+Feira+de+Santana+-+BA,+44090-652,+Brasil/Praia+de+Atalaia,+Aracaju+-+SE,+Brasil/@-11.6003061,-38.6890241,8z/data=!3m1!4b1!4m13!4m12!1m5!1m1!1s0x71437b39667809b:0x8e9dd3a09ee61879!2m2!1d-38.9604431!2d-12.2789361!1m5!1m1!1s0x71ab128b11bbd55:0xedb902da2fa85146!2m2!1d-37.0365845!2d-10.9788554?entry=ttu',
 'descricao_rota': 'Via BR-101'}

# **10 - Cadastrar rota usando o agente**

In [21]:
def cadastrar_rota_cliente_agente():
    """
    Função cliente para cadastrar uma rota selecionando origem e destino da lista.
    """
    print(f"{linha_separadora}\nCadastrar Rota (Agente)\n{linha_separadora}")

    dict_origem = {}
    dict_destino = {}

    while True:
        # A - Listar lugares de origem e hospedagem
        lugares_origem_hospedagem = listar_lugares_finalidade_bd('O') + listar_lugares_finalidade_bd('H')
        if not lugares_origem_hospedagem:
            print("\nAviso: Não há lugares de origem ou hospedagem cadastrados.")
            # Adicionado para evitar loop infinito se não houver lugares
            if not listar_lugares_finalidade_bd('O') and not listar_lugares_finalidade_bd('H'):
                print("Cadastre lugares de origem ou hospedagem antes de criar uma rota.")
                return
        else:
            print("\nSelecione o lugar de ORIGEM:")
            for i, lugar in enumerate(lugares_origem_hospedagem):
                endereco_formatado = retorna_endereco_formatado(lugar)
                print(f"  *{i + 1} (ID {lugar['id']}) - {lugar['descricao']} - {endereco_formatado}")

            # B - Perguntar e armazenar origem
            opcao_origem_str = input("Digite o número da ORIGEM desejada (ou 'S' para voltar ao menu principal): ").upper()
            if opcao_origem_str == 'S':
                print("Saindo do menu de cadastro de rota.")
                return
            try:
                opcao_origem = int(opcao_origem_str)
                if 1 <= opcao_origem <= len(lugares_origem_hospedagem):
                    lugar_origem_selecionado = lugares_origem_hospedagem[opcao_origem - 1]
                    dict_origem['origem_id'] = lugar_origem_selecionado['id']
                    dict_origem['origem_endereco'] = retorna_endereco_formatado(lugar_origem_selecionado)
                    break  # Sai do loop de seleção de origem
                else:
                    print("Opção inválida. Digite um número da lista.")
            except ValueError:
                print("Entrada inválida. Digite um número.")
            # Adicionado para garantir que o loop não continue se não houver lugares válidos.
            if not dict_origem:
                if not lugares_origem_hospedagem: # Se a lista ficou vazia após uma tentativa inválida
                    print("Não há mais opções de origem disponíveis.")
                    return


    while True:
        # C - Listar lugares de destino, hospedagem e visita
        lugares_destino_hospedagem_visita = listar_lugares_finalidade_bd('D') + listar_lugares_finalidade_bd('H') + listar_lugares_finalidade_bd('V')
        if not lugares_destino_hospedagem_visita:
            print("\nAviso: Não há lugares de destino, hospedagem ou visita cadastrados.")
            # Adicionado para evitar loop infinito se não houver lugares
            if not listar_lugares_finalidade_bd('D') and not listar_lugares_finalidade_bd('H') and not listar_lugares_finalidade_bd('V'):
                print("Cadastre lugares de destino, hospedagem ou visita antes de criar uma rota.")
                return
        else:
            print("\nSelecione o lugar de DESTINO:")
            for i, lugar in enumerate(lugares_destino_hospedagem_visita):
                endereco_formatado = retorna_endereco_formatado(lugar)
                print(f"  *{i + 1} (ID {lugar['id']}) - {lugar['descricao']} - {endereco_formatado}")

            # D - Perguntar e armazenar destino
            opcao_destino_str = input("Digite o número do DESTINO desejado (ou 'S' para sair): ").upper()
            if opcao_destino_str == 'S':
                print("Saindo do menu de cadastro de rota.")
                return
            try:
                opcao_destino = int(opcao_destino_str)
                if 1 <= opcao_destino <= len(lugares_destino_hospedagem_visita):
                    lugar_destino_selecionado = lugares_destino_hospedagem_visita[opcao_destino - 1]
                    dict_destino['destino_id'] = lugar_destino_selecionado['id']
                    dict_destino['destino_endereco'] = retorna_endereco_formatado(lugar_destino_selecionado)
                    break  # Sai do loop de seleção de destino
                else:
                    print("Opção inválida. Digite um número da lista.")
            except ValueError:
                print("Entrada inválida. Digite um número.")
            # Adicionado para garantir que o loop não continue se não houver lugares válidos.
            if not dict_destino:
                if not lugares_destino_hospedagem_visita: # Se a lista ficou vazia após uma tentativa inválida
                    print("Não há mais opções de destino disponíveis.")
                    return

    # E - Chamar formata_resposta_agente_buscador_rotas
    try:
        # Verificar se origem e destino foram selecionados
        if not dict_origem.get('origem_endereco') or not dict_destino.get('destino_endereco'):
            print("\nErro: Origem ou destino não selecionado corretamente. Não é possível buscar a rota.")
            return

        resposta_agente = formata_resposta_agente_buscador_rota(dict_origem['origem_endereco'], dict_destino['destino_endereco'])

        # Verificar se a resposta do agente é válida
        if isinstance(resposta_agente, str) and "Erro" in resposta_agente:
            print(f"\nErro ao buscar rota do agente: {resposta_agente}")
            return
        if not isinstance(resposta_agente, dict) or 'distancia_viagem_km' not in resposta_agente or 'link_rota' not in resposta_agente:
            print(f"\nErro: Resposta inesperada do agente buscador de rotas: {resposta_agente}")
            return

        # F - Chamar cadastrar_rota_cliente
        cadastrar_rota_cliente(
            origem_id=dict_origem['origem_id'],
            destino_id=dict_destino['destino_id'],
            quantidade_km=resposta_agente['distancia_viagem_km'],
            link_rota=resposta_agente['link_rota']
        )

    except Exception as e:
        print(f"\nOcorreu um erro ao processar o cadastro da rota: {e}")


# cadastrar_rota_cliente_agente()

# **11 - Menu principal**

In [22]:
def menu_principal():
    """
    Exibe o menu principal e retorna a opção escolhida pelo usuário.
    """
    linha_separadora = "-" * 100
    while True:
        print(f"""
        {linha_separadora}
        Menu Principal
        {linha_separadora}
        1 - Cadastrar lugar
        2 - Alterar lugar
        3 - Exibir lugar de origem
        4 - Exibir lugar de destino
        5 - Exibir lugar de hospedagem
        6 - Listar lugares de visita
        7 - Listar rotas
        8 - Parametrizar sistema
        9 - Exibe valor total de gastos estimados
        10 - Sair
        {linha_separadora}
        """)

        opcao = input("Selecione uma opção: ")

        if opcao in ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10'):
            return opcao
        else:
            print("Opção inválida. Por favor, selecione uma opção válida.")

def main():
    """
    Função principal do programa.
    """
    criar_banco_tabelas()

    print("Bem-vindo ao Viagem Fácil!")
    print("Para começarmos a planejar sua viagem, precisamos de algumas informações.")

    print("\nPrimeiro, precisamos saber de onde você está saindo.")
    cadastrar_lugar_origem_cliente()

    print("\nAgora, informe para onde você deseja ir.")
    cadastrar_lugar_destino_cliente()

    print("\nÓtimo! Agora, diga onde você pretende se hospedar.")
    cadastrar_lugar_hospedagem_cliente()

    print("\nMaravilha! Agora, iremos parametrizar o seu sistema.")
    cadastrar_configuracao_cliente()

    print("\nPerfeito! Com os lugares de origem, destino e hospedagem cadastrados, vamos calcular a melhor rota para sua viagem.")
    cadastrar_rota_cliente_agente()


    while True:
        opcao = menu_principal()

        if opcao == '1':
            cadastrar_lugar_cliente()
        elif opcao == '2':
            alterar_lugar_cliente()
        elif opcao == '3':
            listar_lugar_origem_cliente()
        elif opcao == '4':
            listar_lugar_destino_cliente()
        elif opcao == '5':
            listar_lugar_hospedagem_cliente()
        elif opcao == '6':
            listar_lugar_visita_cliente()
        elif opcao == '7':
            listar_rotas_cliente()
        elif opcao == '8':
            cadastrar_configuracao_cliente()
        elif opcao == '9':
            exibe_gastos_totais_cliente()
        elif opcao == '10':
            print("Saindo do programa.")
            break

# **12 - Executar o programa**

In [ ]:
main()